# DMHR Assignment

## SYSD6 - Leeds 

***

## Importing Useful Libraries Into Python

My first line of code are to import the libraries essential for this project. These include the Numpy, Pandas and Matplotlib libraries for python. 

By importing these libraries as abbreviated terms (e.g. pandas as pd), it is more convinient to access functions and methods fomr them as well as reducing code, making it easier to read when revising or critically evaluating at a later date.

* The numpy library contains many useful methods and functions for mathematical calculations and the manipulation of arrays, for example.


* The Pandas library utilises powerful data structures as well as containing efficient tools for manipulating the data loaded into these. In many cases, manipulation of data using pythons in-built libraries would take minuites, whereas specialised code within the pandas library can carry out what is effectively the same task in a matter of seconds. 


* The Scipy library will be used to conduct linear regressions and provide descriptive statistics to populate visualisations with.


* The matplotlib library contains functions that contribute to a highly flexible and effective platform for the visualisation of data, it can be used in conjunction with other libraries, booth in-built and imported into python (Pandas also incorporates some features from matplotlib). Overall, this provides data scientists with the tools required for creating plots that are  visually pleasing, can convey insights from many differnt types of data and analyse datasets larger than is practical or realistically possible with other commonly used software.
    * '%matplotlib inline' instructs python to produce plots without opening a popup.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
%matplotlib inline


By using an exclamation mark before code, commands form elsewhere in the system can be accessed. The `pip install` command can install external libraries such as `pandasql`.

To install PandaSQL: !pip install pandasql

In [2]:
from pandasql import PandaSQL
pdsql = PandaSQL()


## Reading in the dataframes for this investigation

Reading in first 100 rows of the prescribing dataset in order to guage which columns i will be needing, based on my objectives. 

In [3]:
# Assigning URL to variables as a string, for use
# in reading into pandas dataframes.
url_prescription_data = \
    'https://s3.eu-west-2.amazonaws.com' +\
    '/dmhr-data/prescribing_Dec2015.csv'
    
url_practice_data = \
    'https://s3.eu-west-2.amazonaws.com' +\
    '/dmhr-data/practices_Dec2015.csv'


In [ ]:
# Read first 100 rows of prescribing data and present information
# about columns.

pd.read_csv(url_prescription_data, nrows=100).info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 SHA                                            100 non-null object
PCT                                             100 non-null object
PRACTICE                                        100 non-null object
BNF CODE                                        100 non-null object
BNF NAME                                        100 non-null object
ITEMS                                           100 non-null int64
NIC                                             100 non-null float64
ACT COST                                        100 non-null float64
QUANTITY                                        100 non-null int64
PERIOD                                          100 non-null int64
                                                100 non-null object
dtypes: float64(2), int64(3), object(6)
memory usage: 8.7+ KB


I will only need: 

* Practice codes
* Drug names
* Number of prescriptions
* Actual cost of prescriptions
* Quantity of units of a drug prescribed

These correspond to columns with index: 2, 4, 5, 7 and 8.

By including the option `usecols` as equal to a list of the column indexes nessesary, pandas will only read in and create a dataframe restricted to those columns. This will use less RAM as well as allowing the data to be read in faster.

In [ ]:
# Assign variable to GP prescribing data, select list of
# columns read to dataframe.

prescribing_data_all = \
    pd.read_csv(url_prescription_data,
                usecols=[2, 4, 5, 7, 8])


Reading in the fist 10 rows of the practice dataset shows that there are not column headers. Also there is a final column with no header or contents.

In [ ]:
# Read in first 10 rows from URL and display first 5
pd.read_csv(url_practice_data,
            nrows=10).head()


Reading in only the column headers further reveals that the cells of this dataframe contain large ammounts of whitespace. removing this will reduce memory usage as well as making the data easier to handle.

As this dataset is relatively small I will not exclude any columns, except for the first column, containing the date and the final empty column.

In [ ]:
# Read in first 10 rows from URL and display column headers
pd.read_csv(url_practice_data,
            nrows=10).columns


I have manually added column headers based on the information provided within them.

In [4]:
# Assigning list of column headers for the dataframe to
# to be read in.
practice_columns = ['practice_code', 'practice_name',
                    'building_name', 'street',
                    'city', 'region', 'post_code']

# Read in GP practice dataset into pandas dataframe
# with practice_columns list as headers.
practice_data_all = \
    pd.read_csv(url_practice_data,
                usecols=[1,2,3,4,5,6,7])
    
practice_data_all.columns = practice_columns

practice_data_all.head(2)


,practice_code,practice_name,building_name,street,city,region,post_code
0,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
1,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB


## Cleaning the data

I have defined 3 functions, for the cleaning of these datasets: 
* The first function strips whitespace out of columns headers. 
* The second function strips whitespace form all objects in the data frame, provided they are of the string object type.
* The third and final function comines the previous two.

In [ ]:
# Defining a function to remove whiespace form column headers, if
# column header cell contains a string.
def col_tidy(data_frame):
    new_data_columns = []
    for i in data_frame.columns:
        if type(i) == str:
            i = i.strip()
            i = i.lower()
        new_data_columns.append(i)
    data_frame.columns = new_data_columns
    
# Defnining a function to strip whitespace from cells in dataframe
# if those cells contain objects.
# My laptop struggled with object_tidy. It may not be the most 
# efficient code for this purpose.
# An .apply(Lambda) function would be more efficient in this case
def object_tidy(data_frame):
    for i in data_frame:
        if data_frame[i].dtype == object:
            data_frame.loc[:, i] = \
                (data_frame.loc[:, i]).str.strip()
            data_frame.loc[:, i] = \
                (data_frame.loc[:, i]).str.lower()           

# Defining a function that combines the previous two functions to
# clean both column headers and cells form whitespace.
def dataframe_clean(data_frame):
    col_tidy(data_frame)
    object_tidy(data_frame)


The col_tidy function was used on the prescribing data dataframe as the columns contained a large quantity of whitespace. This was simply done to aid my manipulation of this data.

In [ ]:
col_tidy(prescribing_data_all)
prescribing_data_all.columns


## Subsetting to create new dataframes for the prescription data of Leeds and Cambridge.

One of the first methodologicl problems I have encountered is the definition of which regions consititute as being part of the cities Leeds and Cambridge. Both cities have distinct postcodes, LS and CB, respectively. However, these postcodes extend far beyond what reasonably be considered areas within the two cities. 

Furthermore, if I subset the practice data by rows, for which the region variable contains the city in question it often picks up satellite towns. These towns may or may not be included in Leeds, depending on the definition used. For example, Morely and Bramley, the first two towns in the subset below are towns outside of Leeds. Despite this they are included in the metropolitan borough of Leeds. 

As I have not been briefed on what definition to judge the boundaries of these cities by, I have decided that I will include all practices, for which the city name is included in the city or region. Furthermore, though I will not be using the postcode to filter these out, it will be used as a sanity test to remove any practices that are not in that region of the country. 
* E.g. Some rows in the subset created from the keyword 'leeds' in both the region and city columns gave pracices which were in the PE (Peterborough) postcode area. These will be filtered out.

To do this, I will use .str.contains to assign a boolean value to True to rows containing the city name in either ('|') the region or city columns. case = False, to find strings that contain this word, regardless of case.

The further filtering done by using .str.contains() to remove rows that do not contain the relevant postcode may include irrelevant postcodes if the combination of letters occurs in the postcodes second half. I did not encounter this problem, likely due to the new dataframes being relativley small. If this were a problem I could use a regular expression, such as '^' to specify that I want to only find rows for which the postcode contains the letters at the start.

In [ ]:
# Creating a dataframe containing rows that contain the phrase
# 'leeds', case insensitive in either the 'city' or 'region' column
leeds_practices = \
    practice_data_all.loc[practice_data_all['region']\
                          .str.contains('leeds', case=False) | \
                          practice_data_all['city']\
                          .str.contains('leeds', case=False)]

# Further subsetting the leeds practices for rows only containing
# the Leeds postcode as a sanity check.
leeds_practices = \
    leeds_practices.loc[leeds_practices['post_code']\
                            .str.contains('ls', case=False)]

# Process is repeated for Cambridge.
cambridge_practices = \
    practice_data_all.loc[practice_data_all['region']\
                          .str.contains('cambridge', case=False) | \
                          practice_data_all['city']\
                          .str.contains('cambridge', case=False)]
    
cambridge_practices = \
    cambridge_practices.loc[cambridge_practices['post_code']\
                            .str.contains('cb', case=False)]


As the dataframes contained fewer than 200 rows, The max displayed rows and columns were increased to manually check all observations. Here, only the top 5 are displayed as an example.

In [ ]:
# Setting options of pandas to display more rows and columns.
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
leeds_practices.head()


In [ ]:
cambridge_practices.head()


After creating subsets of the practice dataframe, containing practice data for Leeds and Cambridge, I will use practice codes from these to filter prescribing data for the practices.

* Firstly, I will create lists fo the preactice codes for both these cities.
* Next, I will subset the prescribing dataframe based on rows containing these practice codes.

In [ ]:
# Converting the 'practice_code' series of both dataframes into a list.
cambridge_practice_codes = list(cambridge_practices['practice_code'])
leeds_practice_codes = list(leeds_practices['practice_code'])


In [ ]:
# Subsetting prescribing data based on rows, for which practice code appears
# practice code list of its city. The series are parssed as strings with 
# .contains() function used, case insensitivity is not requuired.
# '|' regex used to join practice lists, as and OR logical.
prescribing_data_leeds = \
    prescribing_data_all.loc[prescribing_data_all['practice']
                             .str.contains('|'.join(leeds_practice_codes))]

prescribing_data_cambridge = \
    prescribing_data_all.loc[prescribing_data_all['practice']
                             .str.contains('|'.join(cambridge_practice_codes))]


***

# Assignment A

## 1. 
By grouping the prescribing data by practice code and applying the .sum() function to it, a new datframe can be made. Here, each practice is reduced to a single row, with the index being its practice code and each column containing the sum of each variable for that practice.

Then, by setting the axis as the practice code, fo the practice dataframes of each city, they can be merged with the new prescribing dataframes for their respective cities.

This produces a dataframe with the details of each practice and its total number of prescriptions, total cost of prescriptions and the total number of units of drugs prescribed.

**The leeds_merged dataframe contains a list of all GP practices in the city of Leeds, along with their total spend on prescriptions ('act cost') and number of unique prescriptions made ('items').**

In [ ]:
# Grouping prescribing datafranes by unique objects in 'practice'
# series with integer and float variables summed.
prescribing_data_cambridge_group = \
    prescribing_data_cambridge.groupby('practice').sum()
    
prescribing_data_leeds_group = \
    prescribing_data_leeds.groupby('practice').sum()


In [ ]:
# Change index of practice dataframes to practice code to allow for
# merging with prescribing dataframes.
cambridge_practices.set_index('practice_code', inplace=True)
leeds_practices.set_index('practice_code', inplace=True)


In [ ]:
cambridge_merged = \
    pd.merge(left=cambridge_practices, \
             right=prescribing_data_cambridge_group, \
             left_index=True, right_index=True)
    
leeds_merged = \
    pd.merge(left=leeds_practices, \
             right=prescribing_data_leeds_group, \
             left_index=True, right_index=True)
    
leeds_merged.head(10)


## 2.


The `groupby()` function in pandas can take a column header as an argument and group each unique value in that column into a single row. Here it is used for the `'bnf name'` column, with associated values added up in each of the new rows.

In [ ]:
# Grouping prescribing data by drug names. Variables within
# groups added together.
presscribing_data_grouped_drugs_leeds = \
    prescribing_data_leeds.groupby('bnf name').sum()


The top 10 most prescribed drugs across all practices in England:

In [ ]:
# Sort dataframe by number of items in descending order.
presscribing_data_grouped_drugs_leeds.sort_values(by='items', 
                                                  ascending=False).head(10)


A list containing the 10 least prescribed drugs would offer little value as there were 1648 drugs prescribed only once in December 2015 in England.

This list may contain different doses of the same drug so it may not accurately how much that particular drug was prescribed in total however.

In [ ]:
# Returns the length of series, for which variable in items
# column is equal to 1.
len(presscribing_data_grouped_drugs_leeds.loc\
    [presscribing_data_grouped_drugs_leeds['items'] == 1])


The total cost of the 10 most prescribed drugs in December 2015 was £361,109.49
The total cost of the 1648 drugs only prescribed once was £135,290.24

In [ ]:
# Returns the sum of cost for the largest 10 values for items.
sum(presscribing_data_grouped_drugs_leeds\
    .sort_values(by='items', ascending=False).head(10)['act cost'])


In [ ]:
# Returns the sum of the drugs only prescribed once.
sum(presscribing_data_grouped_drugs_leeds.loc\
    [presscribing_data_grouped_drugs_leeds['items'] == 1]['act cost'])


The total spend on prescriptions by all practices in Leeds during this time period is £10,152,241.24
The mean spend was £67,681.60 and the median spend was £48,660.76

In [ ]:
# the total spend of all leeds practices
sum(leeds_merged['act cost'])


In [ ]:
# Returns the mean of cost for all practices in the Leeds
# merged practice dataframe.
np.mean(leeds_merged['act cost'])


In [ ]:
# Returns the median of cost for all practices in the Leeds
# merged practice dataframe.
np.median(leeds_merged['act cost'])


The 1648 least prescribed drugs and 10 most prescribed drugs Leeds accounted for 1.33% and 3.56% of spending, repectively, during December of 2015  

In [ ]:
sum(presscribing_data_grouped_drugs_leeds.loc\
    [presscribing_data_grouped_drugs_leeds['items'] == 1]\
    ['act cost'])/sum(leeds_merged['act cost'])*100


In [ ]:
sum(presscribing_data_grouped_drugs_leeds\
    .sort_values(by='items', ascending=False).head(10)['act cost'])\
    /sum(leeds_merged['act cost'])*100


## 3.

To show the 10 most expensive prescriptions made in Leeds during December 2015, I first created a new column containing the average cost of each drug prescribed. This values in this column were set as equal to the division of the actual cost by items.

This list may not reflect the cost of the drugs themselves. E.g. the most expensive prescription made in Leeds during this time period is for 'Tocoph Acet_Tab Chble 100mg'. However, 1080 tablets were included in this prescription. Large prescriptions may misrepresnt the cost of what may otherwise cheap drug.

Alternatively I could work out the cost per unit of drug in a prescription. This, however, may also misrepresent the cost of drugs in the data as some drugs, such as methadone are administed in small incraments of 1ml. Due to this, without understanding the standardised dosage per patient, per time period, using 'quantity' to calculate the most/least expensive drugs may cause similar problems.

In [ ]:
# Creating a new column, where each variable is total cost/prescriptions
# to give mean cost per prescription of drug.
presscribing_data_grouped_drugs_leeds['cost_per_drug'] = \
    presscribing_data_grouped_drugs_leeds['act cost'] \
    /presscribing_data_grouped_drugs_leeds['items']


With this new column, the dataframe could be sorted by by the cost of the drug in a descending order, with the first 10 values being showed.

In [ ]:
# Returns a dataframe with each row representing a drug,
# sorted by mean cost per prescription of drug, in descending order.
presscribing_data_grouped_drugs_leeds.sort_values('cost_per_drug',
                                                  ascending=False).head(10)


## 4.

The average GP practice in Cambridge porduces 29% more prescriptions than the average Leeds practice. As illustrated in the boxplot bellow, this appears to be due to a large number of practices in leeds that made very few prescriptions in December 2015. 

The values for the upper quartiles of prescriptions given out in Leeds and Cambridge GP practices were 14120 and 14512 respectively. This indicates that much of the difference between prescribing between the two cities does come from a large number of practices in Leeds that do not make many prescriptions as opposed to a substantial number of practices in Cambridge that provide a larger than average number of prescriptions.

In [ ]:
print('Mean number of prescriptions made in Leeds GP preactices: %.2f'
      % np.mean(prescribing_data_leeds_group['items']))
print('Mean number of prescriptions made in Cambridge GP preactices: %.2f'
      % np.mean(prescribing_data_cambridge_group['items']))

print('\nMean spend per prescription in Leeds GP preactices: £%.2f'
      % (np.mean(prescribing_data_leeds_group['act cost'])\
      / np.mean(prescribing_data_leeds_group['items'])))
print('Mean spend per prescription in Cambridge GP preactices: £%.2f'
      % (np.mean(prescribing_data_cambridge_group['act cost'])\
      / np.mean(prescribing_data_cambridge_group['items'])))


In [ ]:
# Creates a dictionary for font options in plot.
bold_dict = {'size': 14,
             'weight': 'bold'}

# Returns matplotlib boxplot comparing the distribution of GP practices
# in Cambridge and Leeds by number of prescriptions made
plt.boxplot([prescribing_data_leeds_group['items'],
             prescribing_data_cambridge_group['items']],
            labels = ['Leeds', 'Cambridge'],
            vert = False,
            widths = 0.6)
plt.title('Distribution of Quantity of Prescriptions\nMade by GP Practices ' +
          'In Cambridge and Leeds', fontdict=bold_dict)
plt.xlabel('Number of Prescriptions per Practice', fontdict=bold_dict)
plt.gcf().set_size_inches(11,4)


In [ ]:
print('The 75th percentile of prescriptions per practice for Leeds was: %d'
      % np.percentile(prescribing_data_leeds_group['items'], 75))
print('The 75th percentile of prescriptions per practice for Cambridge was: %d'
      % np.percentile(prescribing_data_cambridge_group['items'], 75))


The distribution of GP practices relative cost per prescription, for both Leeds and Cambridge differ in that there is a greater range for Leeds practices. There are also several outliers in the Leeds practices, with a high spend per prescription that skew the data.

In [ ]:
# Creates a numpy array called <city>_spend_prescription, where each
# each value is the relative cost per prescription for a practice.
leeds_spend_prescription = \
    np.array(prescribing_data_leeds_group['act cost'] /
             prescribing_data_leeds_group['items'])

cambridge_spend_prescription = \
    np.array(prescribing_data_cambridge_group['act cost'] /
             prescribing_data_cambridge_group['items'])


In [ ]:
# Plots a histogram representing the distribution of the relative costs
# per prescription in Leeds and Cambridge.
# Arranged in to 50 bins of equal width.
# Data is normalised as the absolute frequency of Leeds is higher
# due to having more data.
bold_dict = {'size': 14,
             'weight': 'bold'}
plt.hist([leeds_spend_prescription, cambridge_spend_prescription],
         50, alpha=0.85,  color=['b', 'r'],
         label=['Leeds', 'Cambridge'], normed=True)
plt.title('Distribution of Relative Cost Per Prescription for\nGP Practices ' +
          'In Cambridge and Leeds', fontdict=bold_dict)
plt.legend()
plt.xlabel('Relative cost per prescription (£)', fontdict=bold_dict)
plt.ylabel('Frequency of GP practices', fontdict=bold_dict)

plt.gcf().set_size_inches(11,6)


In [ ]:
# Horizontal boxplot, illustrating distribution of <city>_spend_prescription
# arrays for Leeds and Cambridge data.
bold_dict = {'size': 14,
             'weight': 'bold'}
plt.boxplot([leeds_spend_prescription, cambridge_spend_prescription],
            labels=['Leeds', 'Cambridge'],
            vert=False, widths=0.6)
plt.title('Distribution of Relative Cost Per Prescription for GP Practices\n' +
          'In Cambridge and Leeds', fontdict=bold_dict)
plt.xlabel('Relative cost per prescription (£)', fontdict=bold_dict)
plt.xlim(0,20)
plt.gcf().set_size_inches(11,4)


## 5.

In this SQL query, practice codes were counted by grouped cities and regions, as many practices in the `practice_data_all` dataframe have their city value stored as a region and vice versa. Practices are then ordered by their count in a descending order.

For both practice lists grouped by city and region, the modal category was a blank string, i.e. there were more practices, for which this data wsa missing than the next highest city/region. To show the top 10 in descending order, index based positions were used to take a slice of indexes 1 to 10, with a all columns (city first, then count). Simply removing the `.iloc` function form this code will produce a table of practice counts from this query.

In [5]:
print(pdsql("SELECT COUNT(practice_code), city\
       FROM practice_data_all\
       GROUP BY city\
       ORDER BY COUNT(practice_code)\
       DESC", locals()).iloc[1:11, [1, 0]])


                         city  COUNT(practice_code)
1   BIRMINGHAM                                  173
2   LIVERPOOL                                   137
3   LEEDS                                       122
4   MANCHESTER                                  117
5   SHEFFIELD                                   105
6   BRISTOL                                      95
7   COVENTRY                                     95
8   LONDON                                       92
9   BRADFORD                                     85
10  LEICESTER                                    84


In [6]:
print(pdsql("SELECT COUNT(practice_code), region\
       FROM practice_data_all\
       GROUP BY region\
       ORDER BY COUNT(practice_code)\
       DESC", locals()).iloc[1:11, [1, 0]])


                       region  COUNT(practice_code)
1   LONDON                                      869
2   ESSEX                                       410
3   KENT                                        398
4   LANCASHIRE                                  339
5   WEST MIDLANDS                               314
6   SURREY                                      278
7   MIDDLESEX                                   245
8   WEST YORKSHIRE                              234
9   CHESHIRE                                    213
10  HAMPSHIRE                                   184


***








# Assignment B

## 1.

By merging national prescribing data (grouped by practice code) with national practice data, the new `all_practices_merged` dataframe contains the total spending for each practice in England, under the 'act cost' column header.

In [ ]:
# Creating a new dataframe from prescribing data, grouped by
# practice, with numberical variables within group added.
prescribing_data_all_grouped = \
    prescribing_data_all.groupby('practice').sum()

# Merging this new dataframe with practice data(with practice
# code as its index).
all_practices_merged = \
    pd.merge(left=practice_data_all.set_index('practice_code'),
            right = prescribing_data_all_grouped,
            left_index=True, right_index=True)

all_practices_merged.head()


In [ ]:
print('The total number of prescriptions given out in England during December 2015'
       + ' is: ' + str(sum(prescribing_data_all_grouped['items'])) 
      
      + '\n\nThe total spent on prescriptions during this time period is: £'
      + str(sum(prescribing_data_all_grouped['act cost']))
     
      + '\n\nThe mean number of prescriptions given out by GP practices in England was: ' 
      + str(int(np.mean(prescribing_data_all_grouped['items'])))
      
      + '\n\nThe median number of prescriptions given out by GP practices in England was: '
      + str(np.median(prescribing_data_all_grouped['items']))
     )


## 2.

In order to calculate the mean cost per patient for each practice, the HSCIC GP demographic information dataset was imported intp a Pandas data frame. Setting the index of this dataframe as being the practice code, allowed for it to be easily merged with the practice dataframe (also with practice codes as its index).

The all_practices_with_demo dataframe

In [ ]:
# Reading the GP practice demographic data as a pandas dataframe 'demo_data'
# with GP practice code as index.
demo_data = pd.read_csv('https://digital.nhs.uk/media/28273/Numbers-of-' +
                        'Patients-Registered-at-a-GP-Practice-Jan-2016-GP-' +
                        'Practice-and-quinary-age-groups/Any/gp-reg-patients' +
                        '-prac-quin-age', index_col=[0])


In [ ]:
# Merging GP practice dataframe with demographic data.
all_practices_with_demo = \
    pd.merge(left=all_practices_merged,
            right = demo_data, 
            left_index=True, right_index=True)
    
all_practices_with_demo.head()


Many of the columns from the practice and demographic data are not informative when investigating the total number of patients registered at each practice and their average cost of prescriptions. By removing unecessary columns, the data is clearer and easier to interpret.

Using the `np.r_` method to concatenate a combination of indexes and slices, the `all_practices_cost_per_patient` variable was reassigned to a copy of itself, excluding irrelevant columns. This left only columns 0, 3, 5, 6, 7, 8, 16

In [ ]:
# Subsetting the practice + demographic dataframe, to only include
# variables relevant to this analysis. Assigning this subset
# to 'all_practices_cost_per_patient'
all_practices_cost_per_patient = \
    all_practices_with_demo.iloc[:, np.r_[0, 3,5:9,16]]
all_practices_cost_per_patient.head()


In [ ]:
# Dividing cost by number of registered patitents to return a series
# of relative cost per patient for corresponding GP Practices.
# Assigned to new column 'cost_per_patient'
all_practices_cost_per_patient['cost_per_patient'] = \
    all_practices_cost_per_patient['act cost'] \
    /all_practices_cost_per_patient['Total_All']

all_practices_cost_per_patient.head()


In [ ]:
print('The mean cost of prescriptions per patient in England during December 2015 was: £%.2f' \
      % np.mean(all_practices_cost_per_patient['cost_per_patient']))


## 3.

To compare the cost per patient in Leeds GP practices, the all_practices_cost_per_patient dataframe was reindexed to move practice codes into a column and then subsetted based on rows that contained practice codes in Leeds.

Both dataframes had their indexes reset to be the practice codes for each practice.

There is a very strong correlation between number of patients registered at a GP practice and how much it spends on prescriptions for both England as a whole (r² = 0.769) and Leeds (r² = 0.560). The national correlation coefficient is higher than that of Leeds, though this could be to more data points reducing the effect of outliers.

The `stats.linregress()` function of `scipy` returns a special type of list, containing descriptive statistics on the linear regression of two variables from a dataset.

In [ ]:
all_practices_cost_per_patient.reset_index(inplace=True)

leeds_practices_cost_per_patient = \
    all_practices_cost_per_patient.loc[all_practices_cost_per_patient['index']
                                       .str.contains
                                       ('|'.join(leeds_practice_codes))]

all_practices_cost_per_patient.set_index('index', inplace=True)
leeds_practices_cost_per_patient.set_index('index', inplace=True)


In [ ]:
# Assigning the 'get current figure' function of matplotlib as axes
axes = plt.gcf()

bold_dict = {'size': 14,
             'weight': 'bold'}

# Creating two scatter plots, with total number of registered patients
# per practice, plotted against total spend per practice.
plt.scatter(all_practices_cost_per_patient['Total_All'],
            all_practices_cost_per_patient['act cost'],
            marker='o')
plt.scatter(leeds_practices_cost_per_patient['Total_All'],
            leeds_practices_cost_per_patient['act cost'],
            color='r', marker='o')

plt.legend(['England', 'Leeds'])
plt.ylim(0,600000)
plt.xlim(0,40000)
plt.title('The Relationship Between Spending and ' +
          'Number of Registered Patients', fontdict=bold_dict)
plt.xlabel('Number of Registered Patients', fontdict=bold_dict)
plt.ylabel('Total Cost of Prescriptions', fontdict=bold_dict)

axes.set_size_inches(15,9)


In [ ]:
# stats.linregress()[2] returns the correlation coefficient for
# these data. Their exponents of 2 gives the r² values.
regress_england = \
    stats.linregress(all_practices_cost_per_patient['Total_All'],
                     all_practices_cost_per_patient['act cost'])\
                     [2] ** 2

regress_leeds = \
    stats.linregress(leeds_practices_cost_per_patient['Total_All'],
                     leeds_practices_cost_per_patient['act cost'])\
                     [2] ** 2

print('England r² = %.3f\nLeeds r² = %.3f' % (regress_england, regress_leeds))


## 4.

In [ ]:
axes = plt.gcf()

bold_dict = {'size': 14,
             'weight': 'bold'}

plt.hist(all_practices_cost_per_patient['cost_per_patient'],
         range=(5, 30), bins = 30, color='#E67E22', alpha=0.8)

plt.title('Relative Costs Per Patient In GP Practices In England',
          fontdict=bold_dict)
plt.xticks(range(5,31))
plt.xlabel('Relative costs per patient (£)', fontdict=bold_dict)
plt.ylabel('Number of practices', fontdict=bold_dict)
plt.ylim(0,800)
axes.set_size_inches(15,10)


## 5.

To create a Leeds specific dataframe containing basic information about practices as well as their demographic data, a subset was created from the all_practices_with_demo dataframe. The .ix() method was used to select rows, for which the practice code contained within was of a Leeds GP practice.

Alomost all of the practices with codes that began with the letter 'Y' were not referenced in the demographic data, therefore the columns could not previously have been merged and they contain Not a Number (NaN) as each variable.

To remove practices with no data, the .dropna() method will be used. As the columns with missing data are missing data from all columns, I will tell the function to only drop rows for which all values are missing, so as not to remove any rows that might only be missing a small number of variables. 

In [ ]:
leeds_practices_with_demo  = \
    all_practices_with_demo.ix[leeds_practice_codes]
leeds_practices_with_demo.head(10)


In [ ]:
leeds_practices_with_demo.dropna(axis=0, how='all', inplace=True)
leeds_practices_with_demo.head(10)


There is no descernable difference in the ratio of males to females, when comparing GP practices in Leeds as compared to those in England as a whole.

In [ ]:
england_male = sum(all_practices_with_demo['Total_Male'])
england_female = sum(all_practices_with_demo['Total_Female'])

england_male_proportion = england_male/(england_female+england_male)
england_female_proportion = england_female/(england_female+england_male)

leeds_male = sum(leeds_practices_with_demo['Total_Male'])
leeds_female = sum(leeds_practices_with_demo['Total_Female'])

leeds_male_proportion = leeds_male/(leeds_female+leeds_male)
leeds_female_proportion = leeds_female/(leeds_female+leeds_male)

axes = plt.gcf()
fig, ax = plt.subplots(figsize=(5,7))
x_ax = np.arange(2)
width = 0.3
b1 = ax.bar(x_ax, [england_male_proportion, leeds_male_proportion], width, label='Male')
b2 = ax.bar(x_ax + width, [england_female_proportion, leeds_female_proportion] , width, label='Female')

plt.ylim(0.4, 0.6)
plt.yticks([0.4, 0.45, 0.5, 0.55, 0.6])
plt.ylabel('Percentage')
plt.legend()


To create histograms of the age distribution for GP practices in Leeds and England, as a whole 

The original bins for the age groups in the demographic data was sepperated by sex. To summarise the age distribution of all people, the values for each sex were added together and new bins were created, without reference to sex. These new bins were systematically created using a for loop to save time and prevent typing errors.

In [ ]:
a = 0
age_bins = []
for i in range(20):
    age_bins.append(a)
    a += 5
age_bins


By manually creating bins, I have created a histrogram of the age distribution for GP practices in Leeds, using the bar chart function of matplotlib.

In [ ]:
leeds_male_demographics = \
    pd.DataFrame(leeds_practices_with_demo.sum().iloc[19:39])
leeds_female_demographics = \
    pd.DataFrame(leeds_practices_with_demo.sum().iloc[39:])

leeds_male_and_female_ages = \
    np.array(leeds_male_demographics[0]) + \
    np.array(leeds_female_demographics[0])


leeds_age_demographics = pd.DataFrame(leeds_male_and_female_ages, age_bins)

all_male_demographics = \
    pd.DataFrame(all_practices_with_demo.sum().iloc[19:39])
all_female_demographics = \
    pd.DataFrame(all_practices_with_demo.sum().iloc[39:])

all_male_and_female_ages = \
    np.array(all_male_demographics[0]) + \
    np.array(all_female_demographics[0])


all_age_demographics = pd.DataFrame(all_male_and_female_ages, age_bins)

bold_dict = {'size': 14,
             'weight': 'bold'}

plt.bar(list(all_age_demographics.index), all_age_demographics[0], 5,
        align='edge', color='#FA8072')

plt.xlim(0, 100)
plt.xticks(range(0, 101, 5))
plt.title('Age Distribution In of Registered Patitents at Leeds GP Practices',
          fontdict=bold_dict)
plt.xlabel('Age', fontdict=bold_dict)
plt.ylabel('Frequency', fontdict=bold_dict)

plt.gcf().set_size_inches(9, 6)


***

# Assignment C

## 1.

Assuming that one prescription of a statin represents a patient, and that each patient is only prescribed one type of statin, once a month. This however causes some statins to be represented as being dispraportionatley expensive. For example, the liquid suspensions of atorvastatin, are prescribed as large quantities of doses in very few prescriptions.

The demographic data used was recorded for the quarter, ending in January 2016.

In [ ]:
england_patients = \
    sum(demo_data['Total_All'])
    
print('In total, there were %d patients in England ' +
      'at the time of Janury 2016.' % england_patients)


In [ ]:
# List of statin drugs to search prescribing data for.
statin_drugs = ['simvastatin', 'atorvastatin',
                'rosuvastatin', 'pravastatin',
                'fluvastatin']

# Subset dataframe to only statins
statin_prescriptions = \
    prescribing_data_all.loc[prescribing_data_all['bnf name']
                             .str.contains('|'.join(statin_drugs), case=False)]

# Group statin prescription dataframe by drug name.
statin_prescriptions = \
    statin_prescriptions.groupby('bnf name').sum()

statin_prescriptions.head()


**Statin Prescriptions:**

In [ ]:
# Calculating the relative cost per prescription
statin_prescriptions['relative_cost_per_prescription'] = \
    statin_prescriptions['act cost']/statin_prescriptions['items']

# Calculating the relative cost per patient
statin_prescriptions['relative_cost_per_patient'] = \
    statin_prescriptions['act cost']/england_patients

# Renaming columns for presentation
statin_prescriptions.columns = ['Number of Prescriptions',
                                'Total Spend on Drug (£)',
                                'Total Units of Drug Prescribed',
                                'Relative Cost of Drug per Prescription (£)',
                                'Relative Cost of Drug per Patient (£)']

# Round the columns containing monetery values to 2 decimal places
money_columns = ['Total Spend on Drug (£)',
                 'Relative Cost of Drug per Prescription (£)',
                 'Relative Cost of Drug per Patient (£)']

for i in money_columns:
    statin_prescriptions[i] = statin_prescriptions[i]\
                              .apply(lambda x: np.round(x, decimals=2))

statin_prescriptions


In [ ]:
print('For GP practices in England, during the month of December 2015:\n\n')

print('The total number of statin prescriptions was: %d \n'
      % statin_prescriptions['Number of Prescriptions'].sum())

print('The total spend on statin prescriptions was: £%d \n'
      % statin_prescriptions['Total Spend on Drug (£)'].sum())

print('The mean cost per prescription of a statin was: £%.2f'
      % np.mean(statin_prescriptions['Relative Cost of Drug per Prescription (£)']))
print('The median cost per prescription of a statin was: £%.2f \n'
      % np.median(statin_prescriptions['Relative Cost of Drug per Prescription (£)']))

print('The total cost of statins per patient in England was: £%.2f \n'
      % statin_prescriptions['Relative Cost of Drug per Patient (£)'].sum())

statins_percent = \
    (statin_prescriptions['Total Spend on Drug (£)'].sum()\
     /prescribing_data_all['act cost'].sum())*100
statins_percent = np.round(statins_percent, decimals=3)
    
    
print('Statins made up %' + str(statins_percent) + ' of all prescription costs')


## 2.

The deprivation dataset was read into a pandas dataframe, named `imd`. Only the relevant columns of the dataset were imported. These included:
* *'Postcode'* - Set as the index column, for the purpose of merging with GP prescribing data from Leeds and England as a whole.
* *'Index of Multiple Deprivation Rank'* and *'Index of Multiple Deprivation Decile'* - for analysis of their relationship to statin prescriptions.

In [ ]:
# Reading the IMD dataset as pandas dataframe imd
url = \
    'https://s3.eu-west-2.amazonaws.com/dmhr-data/deprivation-by-postcode.csv'

imd = pd.read_csv(url, usecols=['Postcode',
                                'Index of Multiple Deprivation Rank',
                                'Index of Multiple Deprivation Decile'],
                  index_col='Postcode')
imd.head()


The `leeds_practices_cost_per_patient` dataframe (with *practice code* replaced by *post code* as index) was merged with the `imd` dataframe. 

Once merged, the index of the new `leeds_imd` dataframe was reset and replaced with practices codes.

In [ ]:
leeds_imd = pd.merge(left = leeds_practices_cost_per_patient.reset_index().set_index('post_code'),
                    right = imd,
                    left_index = True,
                    right_index = True)
leeds_imd.reset_index(inplace=True)
leeds_imd.set_index('index', inplace=True)
leeds_imd.head(3)


To improve the presentation of the data, column headers and the index header were renamed. The columns were then rearranged using integer based positioning (`.iloc`) and `numpy.r_`, for concatenation of slices of these positions.

In [ ]:
# Renaming columns of leeds_imd dataframe
leeds_imd.columns = ['post code',
                     'practice name',
                     'city',
                     'prescriptions made',
                     'total cost of prescriptions',
                     'units of drug prescribed',
                     'total number of patients',
                     'cost per patient',
                     'index of multiple deprivation rank',
                     'index of multiple deprivation decile']

# Renaming index header.
leeds_imd.index.name = 'practice code'

leeds_imd.head(3)


In [ ]:
# Assigning the leeds_imd dataframe to a concatenation of slices
# of itself, in order to re-order the columns for presentation.
leeds_imd = leeds_imd.iloc[:, np.r_[1 , 2 , 0 , 8:10 , 3:8 ]]
leeds_imd.head()


## 3.

A subset of the national GP prescribing dataframe was created, under the name of `england_statin_prescriptions`, with only the unique prescriptions of statin drugs.

In [ ]:
# Creating a subset of national prescribing data for statin drugs.
england_statin_prescriptions = \
    prescribing_data_all.loc[prescribing_data_all['bnf name']
                             .str.contains('|'.join(statin_drugs), case=False)]

england_statin_prescriptions.head()


Grouping by (`df.groupby()`) *practice* and the use of the *.sum()* method creates a new dataframe that summarises the numerical data for statin prescriptions in each English GP practice for the month of December 2015.

In [ ]:
# Grouping nationwide statin prescriptions by practice.
england_statin_prescriptions = \
    england_statin_prescriptions.groupby(by='practice').sum()

england_statin_prescriptions.head()


In [ ]:
# Merging statin prescription data with demographic data.
england_statin_prescriptions = \
    pd.merge(left=england_statin_prescriptions,
             right=demo_data.iloc[:, np.r_[0, 3, 7]],
             left_index=True,
             right_index=True)

england_statin_prescriptions.head()


In [ ]:
# Merging statin prescription by practice with relevant information
# on practices
england_statin_prescriptions = \
    pd.merge(left=england_statin_prescriptions,
             right=practice_data_all.set_index('practice_code')
                                    .iloc[:, np.r_[0:5]],
             left_index=True,
             right_index=True)

england_statin_prescriptions.head()


In [ ]:
# Setting postcode as index for merginf with IMD data.
england_statin_prescriptions.reset_index(inplace=True)
england_statin_prescriptions.set_index('POSTCODE', inplace=True)

# Strip trailing whitespace from imd index(postcodes)
imd.index = imd.index.str.strip()

# Merging statin prescription data + GP info with IMD data.
england_statin_prescriptions = \
    pd.merge(left=england_statin_prescriptions,
             right=imd,
             left_index=True,
             right_index=True)

england_statin_prescriptions.head()


In [ ]:
# Remove duplicate rows
england_statin_prescriptions.drop_duplicates(inplace=True)

# Set index back to practice codes
england_statin_prescriptions.reset_index(inplace=True)
england_statin_prescriptions.set_index('index', inplace=True)

# Re-ordering columns and renaming headers for better presentation.
england_statin_prescriptions = \
    england_statin_prescriptions.iloc[ : , np.r_[6:11 , 0, 2, 1, 5, 4, 11, 12]]

england_statin_prescriptions.columns = ['practice name' , 
                                       'building name' , 
                                       'street' , 
                                       'city' , 
                                       'region' , 
                                       'post code' , 
                                       'total statin spend (£)' , 
                                       'total statin prescriptions' ,
                                        'total patients registered' ,
                                       'ons region code' , 
                                       'index of multiple deprivation rank' , 
                                       'index of multiple deprivation decile']

england_statin_prescriptions.head()


Two practices do not have any data corresponding to IMD. These have been removed, as they cannot be used to investigate a link between index of multiple deprivation and statin prescriptions. Furthermore, removing NaN values will allow for the conversion of these two series to integer datatypes, as NaN can only be a float value.

In [ ]:
england_statin_prescriptions.loc[~(england_statin_prescriptions['index of multiple deprivation rank'] > 0)]


In [ ]:
# Removing all rows with IMD rank below 0
england_statin_prescriptions = \
    england_statin_prescriptions\
    .loc[england_statin_prescriptions
         ['index of multiple deprivation rank'] > 0]

# Changing data type of IMD rank and decile from float to integer.
england_statin_prescriptions.iloc[:, [10, 11]] = \
    england_statin_prescriptions.iloc[:, [10, 11]].astype(int)

# Create a statin spend per person
england_statin_prescriptions['relative statin spend'] = \
    england_statin_prescriptions['total statin spend (£)']\
    / england_statin_prescriptions['total patients registered']

england_statin_prescriptions.head()


In [ ]:
#lowest relative spenders of statins from the first decile
lowest_decile = \
    england_statin_prescriptions\
    .loc[england_statin_prescriptions['index of multiple deprivation decile'] == 1]

lowest_decile.sort_values('relative statin spend', ascending = True)


In [ ]:
#highest relative spenders of statins from the last decile

highest_decile = \
    england_statin_prescriptions\
    .loc[england_statin_prescriptions['index of multiple deprivation decile'] == 10]

highest_decile.sort_values('relative statin spend', ascending = False)


In [ ]:
# Calculating the mean values for relative statin cost for the
# lowest and highest deciles of IMD
lowest_mean = np.mean(lowest_decile['relative statin spend'])
highest_mean = np.mean(highest_decile['relative statin spend'])

# Calculating the median values for relative statin cost for the
# lowest and highest deciles of IMD
lowest_median = np.median(lowest_decile['relative statin spend'])
highest_median = np.median(highest_decile['relative statin spend'])

print('The mean relative spend on statins, through prescriptions ' +
      'given by GPs:\n\n' +
      'In the first decile of the index of multiple ' +
      'deprivation was: £%.2f \n' % lowest_mean +
      'In the tenth decile of the index of multiple ' +
      'deprivation was: £%.2f \n\n' % highest_mean)

print('The median relative spend on statins, through prescriptions ' +
      'given by GPs:\n\n' +
      'In the first decile of the index of multiple ' +
      'deprivation was: £%.2f \n' % lowest_median +
      'In the tenth decile of the index of multiple ' +
      'deprivation was: £%.2f ' % highest_median)


The difference between the mean values was far greater than that of the median values, suggesting and outlier in the first decile.

This outlier is GP practice E87723, NEW ELGIN PRACTICE in West London. This practice is reocreded as only having a single patient but having made 352 prescriptions for statins.

Using its label based location reveals that this practice was recorded has having a single patient. However, searching the prescribing data for this practice reveals that there were 1104 prescriptions made in Dec 2015. Due to this, it is likely that this outlier is caused by an error in the demographic data. 

As this only affects one practice out of over 7000 ( < 0.02% of the data) I will drop it from the `england_statin_prescriptions` dataframe.

In [ ]:
# Returns dataframe of practices with a large (over £1) relative
# cost of statin prescriptions.
england_statin_prescriptions.loc[england_statin_prescriptions
                                 ['relative statin spend'] > 1]


In [ ]:
# Returns registered patient data for the anomalous practice 'E87723'.
demo_data.loc['E87723', ['Total_Male', 'Total_Female', 'Total_All']]


In [ ]:
# Returns shape precribing data for preactice E87723
prescribing_data_all.loc[prescribing_data_all['practice']
                         .str.contains('E87723')].shape


In [ ]:
# Removing practice E87723 from the statin prescription data.
england_statin_prescriptions.drop('E87723', axis=0, inplace=True)


In [ ]:
lowest_decile = \
    england_statin_prescriptions\
    .loc[england_statin_prescriptions
         ['index of multiple deprivation decile'] == 1]

lowest_mean = np.mean(lowest_decile['relative statin spend'])
lowest_median = np.median(lowest_decile['relative statin spend'])

print('The new mean relative spend on statins ' +
      'in the first decile is: £%.2f \n'
      % lowest_mean +
      'The new median relative spend on statins ' +
      'in the first decile is: £%.2f \n'
      % lowest_median)


There is little difference to the distribution of spending on statin prescriptions between the 10 deciles of IMD, as illustrated by the box plot below.

In [ ]:
# Empty list to loop through
imd_deciles = []

# Creates a list of numpy arrays, with each array containing the
# values for relative statin prescription spending for each decile.
for i in range(1,11):
    imd_deciles.append(np.array(england_statin_prescriptions
                       .loc[england_statin_prescriptions
                            ['index of multiple deprivation decile'] == i]
                            ['relative statin spend']))


In [ ]:
u = []
for i in range(1,11):
    u.append('Decile ' + str(i))
u


In [ ]:
bold_dict = {'size': 14,
             'weight': 'bold'}

plt.boxplot(imd_deciles,
            labels=['Decile 1', 'Decile 2',
                    'Decile 3', 'Decile 4',
                    'Decile 5', 'Decile 6',
                    'Decile 7', 'Decile 8',
                    'Decile 9', 'Decile 10'])

plt.ylim(0,1)
axes = plt.gcf()
axes.set_size_inches(11,8)

plt.yticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5,
            0.6, 0.7, 0.8, 0.9, 1.0])
plt.title('Relatve Statin Spend Per Patient By IMD Decile', fontdict=bold_dict)
plt.ylabel('Relatve Statin Spend Per Patient (£)', fontdict=bold_dict)
plt.xlabel('Decile', fontdict=bold_dict)


Practices that are in the lowest decile of **relative costs of statin prescriptions** tend to be of a lower IMD rank, as illustrated in the boxplot below.

In [ ]:
england_statin_prescriptions.sort_values('relative statin spend',
                                         ascending=True, inplace=True)

x = int(len(england_statin_prescriptions)/10)

top_10_percent_statins = \
    np.array(england_statin_prescriptions
             ['index of multiple deprivation rank'].tail(x))

bottom_10_percent_statins = \
    np.array(england_statin_prescriptions
             ['index of multiple deprivation rank'].head(x))

plt.boxplot([top_10_percent_statins,
            bottom_10_percent_statins],
            labels=['Top Decile', 'Bottom Decile'])

plt.ylabel('IMD Rank', fontdict=bold_dict)
plt.xlabel('Decile of Statin Cost', fontdict=bold_dict)
plt.title('IMD Ranks of The Practices That Have the Highest and\n' +
          'Lowest Relative Cost Of Statin Prescriptions', fontdict=bold_dict)


In [ ]:
np.mean(bottom_10_percent_statins)


In [ ]:
np.mean(top_10_percent_statins)


Combining the lowest and highest spenders of the practices in the catchment area of the least and most deprived areas respectively, gives the `both_spend_groups` dataframe.

In [ ]:
both_spend_groups = \
    pd.concat([lowest_decile.sort_values('relative statin spend', ascending=True).head(100), 
               highest_decile.sort_values('relative statin spend', ascending=False).head(100)],
               axis=0)


## 4.

Importing the *75 mortality rate for cardiovascular diseases* dataset as `cvd75`.

In [ ]:
cvd75 = \
    pd.read_csv('https://s3.eu-west-2.amazonaws.com/dmhr' +
                '-data/NHSOF_1.1_I00656_D.csv')


In [ ]:
cvd75.head()


In order to extract regional data, a `df.column_head.unique()` query was utilised to find which column contained region names and what exactly the names were, in order to create a subset dataframe.

To conduct a query of unique values, all column names changed to not have space. A function to replace spaces with underscores may have been more practical with more column headers. However, as there were relatively few, manually renaming was faster.

In [ ]:
cvd75.columns = ['year', 'period', 'breakdown', 'level',
                 'level_description', 'gender', 'age', 'indicator_value',
                 'lower_ci', 'upper_ci', 'numerator', 'denominator']


In [ ]:
cvd75.level_description.unique()


From the unique values in the level description column, the exact phrases reffering to the nine English regions were copied into a list. This list was used to create a new dataframe (`cvd75_regions`), containing only rows that made reference to each region **and** the year 2015 **and** all genders ('Person')

In [ ]:
nine_english_regions = ['East Midlands', 'East of England', 'London',
                        'North East', 'North West', 'South East', 'South West',
                        'West Midlands', 'Yorkshire and The Humber']


In [ ]:
cvd75_regions_2015 = cvd75.loc[cvd75['level_description'].str.contains
                               ('|'.join(nine_english_regions)) &
                               (cvd75['year'] == 2015) & 
                               (cvd75['gender'] == 'Person')]

cvd75_regions_2015


The *ONS Postcode Lookup* dataset can be used to link GP practices to thier corresponding region, through their postcode. Only the columns at index 2 (postcode) and 16 (region code) were needed, so to save memory, only these two were read into a pandas dataframe.

In [ ]:
postcode = pd.read_csv('https://s3.eu-west-2.amazonaws.com/dmhr-data/postcodes.csv',
                       usecols=[2, 16], index_col=0)
postcode.columns = ['region_code']
postcode.head()


In [ ]:
postcode.region_code.unique()


Now that there is a dataframe that links each postcode to its corresponding region code, it can be seen that there are more unique region codes than contained within the CVD75 dataset. The additional region codes do not follow the same naming convention. 

Rows in the `postcode` dataframe that contained a region code, not foiund in the `cvd75_regions_2015` dataframe were filtered out. Rows wiht NaN values were also removed from the `postcode` dataframe.

In [ ]:
region_codes = list(cvd75_regions_2015['level'])
region_codes


In [ ]:
postcode.dropna(axis=0, inplace=True)

postcode = \
    postcode.loc[postcode['region_code'].str.contains('|'.join(region_codes))]


The `postcode` now only contains rows, for which the region code is one of the nine from the `cvd75_regions_2015` dataframe.

In [ ]:
postcode.region_code.unique()


A new column was created with a lambda function that added a cell with the region name corresponding to the region code of that row. A dictionary was used to match the key (region code) to the value (region name).

As dictionaries are mutable, a loop could have been created to add region codes and thier corresponding names to a dictionary. However, at only 9 key-value pairs, doing this manually would have been more time efficient.

In [ ]:
english_regions = {'E12000004' : 'East Midlands',
                   'E12000006' : 'East of England',
                   'E12000007' : 'London',
                   'E12000001' : 'North East',
                   'E12000002' : 'North West',
                   'E12000008' : 'South East',
                   'E12000009' : 'South West',
                   'E12000005' : 'West Midlands',
                   'E12000003' : 'Yorkshire and The Humber'}

postcode['region_name'] = postcode['region_code'].apply(lambda x: english_regions[x])

postcode.head()


The `postcode` dataframe can now be merged with the `england_statin_prescriptions` dataframe to add information about which of the 9 English regions the GP practices belong to.

In [ ]:
x = len(england_statin_prescriptions)


In [ ]:
england_statin_prescriptions.reset_index(inplace=True)
england_statin_prescriptions.set_index('post code', inplace=True)

england_statin_prescriptions = pd.merge(left = england_statin_prescriptions,
                                         right = postcode,
                                         left_index=True,
                                         right_index=True)

england_statin_prescriptions.reset_index(inplace=True)
england_statin_prescriptions.set_index('index', inplace=True)


In the process of merging the two dataframes, 31 rows were not carried over. This could be due to the rows not having a postcode corresponding to the `postcode` dataframe.

In [ ]:
y = len(england_statin_prescriptions)
x - y


In [ ]:
england_statin_prescriptions.head(1)


In [ ]:
england_statin_prescriptions.columns = ['post code', 'practice name',
                                        'building name', 'street', 'city',
                                        'region', 'total statin spend (£)',
                                        'total statin prescriptions',
                                        'total patients registered',
                                        'ons region code',
                                        'index of multiple deprivation rank',
                                        'index of multiple deprivation decile',
                                        'relative statin spend', 'region code',
                                        'region name']


In [ ]:
england_statin_prescriptions.head()


## 5.

Grouping the `england_statin_prescriptions` dataframe by region code and calculating the mean of their relative spend on statin prescriptions. This dataframe, containing the mean statin spend per region was merged with the `cvd75` dataframe. Finally, only the columns corresponding to mortality rates and relative cost of statin prescriptions were included in the new `cvd75_and_statins` dataframe.

There is a weak correlation (*r²* = 0.15) between spending on statins and CVD75 mortality rates in England. However, practices in London spend substantially less on statin prescriptions than other regions of comparable mortality rates. Due to this, I beleive that there may a confounder influencing the spend on statins per patient. 

When London is excluded from analysis, the correlation between spending on statins and CVD75 mortality rates in England becomes moderate to strong (*r²* = 0.429).

Furthermore the data points appear to cluster based on geographic proximity. With the exception of London, these clusters include:

* The South (East of England, South East, South West)
* The Midlands (East Midlands, West Midlands)
* The North (North East, North West, Yorkshire and The Humber)

Regions that are higher in lattitude (with the exception of London) appear to have significantly\* higher mortality rates. At a glance there does not appear to be as strong a relationship between lattitude and statin spending alone. Further analysis could use coordiantes, linked with practice codes to assess geographical factors and their relationship to statin spending and mortality.

\* Significance from confidence intervals provided in the CVD75 dataset.

In [ ]:
region_statins = \
    england_statin_prescriptions.groupby('region name').mean()
    
cvd75_and_statins = \
    pd.merge(left = cvd75_regions_2015.set_index('level_description'),
             right = region_statins,
             left_index = True,
             right_index = True)

cvd75_and_statins = \
    cvd75_and_statins.iloc[: , [6, 7, 8, 16]]
    
cvd75_and_statins


The indicator values (mortality rate) are stored as strings. They will be converted into floats. This will make them easier to handle when graphing.

In [ ]:
cvd75_and_statins['indicator_value'].dtype


In [ ]:
columns_to_float = ['indicator_value',
                    'lower_ci',
                    'upper_ci']

for i in columns_to_float:
    cvd75_and_statins[i] = cvd75_and_statins[i].apply(lambda x: float(x))
    print(i + ' : ' + str(cvd75_and_statins[i].dtype))


In [7]:
def plot_line(x_vals, y_vals, line_colour, line_style, label_name):
    m = stats.linregress(x_vals, y_vals)[0]
    b = stats.linregress(x_vals, y_vals)[1]
    x_1 = 0.15
    x_steps = 0.01
    x = []
    y = []
    for i in range(16):
        x.append(x_1)
        y_val = ((x_1*m) + b)
        y.append(y_val)
        x_1 += x_steps
    plt.plot(x,y, c=line_colour, ls=line_style, label=label_name, alpha=0.8)

In [ ]:
axes = plt.gcf()

plot_df = cvd75_and_statins
x_val = plot_df['relative statin spend']
y_val = plot_df['indicator_value']

point_labels = list(plot_df.index.values)
point_colour = '#2ECC71'

bold_dict = {'size': 14,
             'weight': 'bold'}

plt.scatter(x_val, y_val, c=point_colour, s=100)

for i, j in enumerate(point_labels):
    plt.annotate(j, (plot_df['relative statin spend'][i],
                     plot_df['indicator_value'][i]))

axes.set_size_inches(15, 9)
plt.title('The Relationship Between Statin Prescriptions\nand ' +
          'The Mortality Rate of Under-75s From\nCardiovascular Disease',
          fontdict=bold_dict)
plt.xlabel('Relative Spend on Statins (£)', fontdict=bold_dict)
plt.ylabel('Under-75 Mortality rate from CVD', fontdict=bold_dict)
plt.xlim(0.15, 0.30)
plt.ylim(50, 100)
plt.yticks([50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100])
plt.xticks([0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 0.22, 0.23, 0.24, 0.25,
            0.26, 0.27, 0.28, 0.29, 0.30])


plot_line(x_val, y_val, 'k', '--', 'Linear Regression')

text_box = 'r² = %.3f' % (stats.linregress(x_val, y_val)[2] ** 2)

axes.text(0.91, 0.65, s=text_box, fontdict=bold_dict)

y_error_vals = ([abs(y_val - plot_df['lower_ci']),
                 abs(y_val - plot_df['upper_ci'])])

plt.gca().errorbar(x_val, y_val, yerr=y_error_vals, fmt='none', c=point_colour)


In [ ]:
y_error_vals


In [ ]:
np.corrcoef(cvd75_and_statins['relative statin spend'],
             cvd75_and_statins['indicator_value'])[1,0]


In [ ]:
stats.linregress(cvd75_and_statins['relative statin spend'],
                cvd75_and_statins['indicator_value'])


When London is exluded from the dataset, a stronger correlation emerges between relative spend on statins and under 75 mortality from CVD. Is London an anomoly?

In [ ]:
cvd_no_london = cvd75_and_statins.loc[cvd75_and_statins.index != 'London']
np.corrcoef(cvd_no_london['relative statin spend'],
             cvd_no_london['indicator_value'])[1,0]


In [ ]:
stats.linregress(cvd_no_london['relative statin spend'],
                cvd_no_london['indicator_value'])


In [ ]:
axes = plt.gcf()

plot_df = cvd_no_london
x_val = plot_df['relative statin spend']
y_val = plot_df['indicator_value']

point_labels = list(plot_df.index.values)
point_colour = 'y'

bold_dict = {'size': 14,
             'weight': 'bold'}

plt.scatter(x_val, y_val, c=point_colour, s=100)

for i, j in enumerate(point_labels):
    plt.annotate(j, (plot_df['relative statin spend'][i],
                     plot_df['indicator_value'][i]))

axes.set_size_inches(15, 9)
plt.title('The Relationship Between Statin Prescriptions\nand ' +
          'The Mortality Rate of Under-75s\nFrom Cardiovascular Disease\n' +
          '(Without Data From London)', fontdict=bold_dict)
plt.xlabel('Relative Spend on Statins (£)', fontdict=bold_dict)
plt.ylabel('Under-75 Mortality rate from CVD', fontdict=bold_dict)
plt.xlim(0.15, 0.30)
plt.ylim(50, 100)
plt.yticks([50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100])
plt.xticks([0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 0.22, 0.23, 0.24, 0.25,
            0.26, 0.27, 0.28, 0.29, 0.30])


plot_line(x_val, y_val, 'r', '--', 'Linear Regression')

text_box = 'r² = %.3f' % (stats.linregress(x_val, y_val)[2] ** 2)

axes.text(0.91, 0.65, s=text_box,
          fontdict=bold_dict)

y_error_vals = ([abs(y_val - plot_df['lower_ci']),
                 abs(y_val - plot_df['upper_ci'])])

plt.gca().errorbar(x_val, y_val, yerr=y_error_vals, fmt='none', c=point_colour)


***

# Assignment D

## 1.

For this assignment, I will be importing Google flu trend data for Germany and Chile. As these datasets are stored in a `.txt` format, I will use the `pandas.read_table()` function. As `.txt` and `.csv` are both 'flat files', the data is parsed in a simmilar manner and can take the same arguments and parsing options. In both datasets, the headers are on the 8th row and the first column contains dates in a format, easily recognised and parsed by pandas. Instead of using the `pandas.to_datetime` function, less code is required to parse the date column as a datetime series during the reading in stage.



In [ ]:
germany_flu = \
    pd.read_table('https://www.google.org/flutrends/' +
                  'about/data/flu/de/data.txt',
                  sep=',', header=8, parse_dates=[0])
germany_flu.set_index('Date', inplace=True)

chile_flu = \
    pd.read_table('https://www.google.org/flutrends/' +
                  'about/data/flu/cl/data.txt',
                  sep=',', header=8, parse_dates=[0])
chile_flu.set_index('Date', inplace=True)


In [ ]:
print('Germany date format: %s\nChile date format: %s'
      % (germany_flu.index.dtype, chile_flu.index.dtype))


In [ ]:
axes = plt.gcf()
date = germany_flu.index.values

# Both the data for flu trends in Germany and Chile were divided
# by their maximum value to normalise the data.
plt.plot(germany_flu['Germany']/max(germany_flu['Germany']))
plt.plot(chile_flu['Chile']/max(chile_flu['Chile']))
plt.legend()


plt.xlim(min(date), max(date))
plt.title('Flu Trends in Germany & Chile', fontdict=bold_dict)
plt.xlabel('Date')
plt.ylabel('Relative Frequency')

axes.set_size_inches(15, 4)


Same plot, with a smoother line:

In [ ]:
axes = plt.gcf()

# Resampling both plots with a trailing mean of
# 2 months to give a smoother line
(germany_flu['Germany']/max(germany_flu['Germany']))\
    .resample('2M', how='mean').plot()

(chile_flu['Chile']/max(chile_flu['Chile']))\
    .resample('2M', how='mean').plot()

plt.legend()
plt.xlim(pd.Timestamp('2004-01-01'), pd.Timestamp('2015-08-01'))
plt.title('Flu Trends in Germany & Chile', fontdict=bold_dict)
plt.xlabel('Date')
plt.ylabel('Relative Frequency')

axes.set_size_inches(15, 4)


## 2.

Grouping the dataframes for Germany and Chile by index year, then applying the `.min()` and `.max()` functions gave the yearly minimum and maximum frequencies by year. Merging These dataframes together gives the anual minimum and maximum frequencies for Germany and Chile.

In [ ]:
germany_minmax = \
    pd.merge(left=pd.DataFrame(germany_flu.groupby
                               ([germany_flu.index.year]).max()['Germany']),
             right=pd.DataFrame(germany_flu.groupby
                                ([germany_flu.index.year]).min()['Germany']),
             left_index=True,
             right_index=True)
germany_minmax.columns = ['Germany Annual Maximum', 'Germany Annual Minimum']

chile_minmax = \
    pd.merge(left=pd.DataFrame(chile_flu.groupby
                               ([chile_flu.index.year]).max()['Chile']),
             right=pd.DataFrame(chile_flu.groupby
                                ([chile_flu.index.year]).min()['Chile']),
             left_index=True,
             right_index=True)
chile_minmax.columns = ['Chile Annual Maximum', 'Chile Annual Minimum']

all_minmax =\
    pd.merge(left=germany_minmax,
             right=chile_minmax,
             left_index=True,
             right_index=True)

all_minmax


As the flu trends for Germany are more consistent, year on year, I will be using this as the basis for my model. Furthermore, I will use the 2 month trailing mean of Germany flu trends as a benchmark for my model. This is due to this transformtation showing more general seasonal trends, less affected by sudden spikes, for which I will not be aiming to predict.

Each increment of 1 on the x axis of my model will correspond to a single week.

In [ ]:
x = np.arange(0.5, 140.6, 2)
y = (np.sin(x/1.9 + 1.5) + 1.9) / 4

model_1 = pd.DataFrame(y, x)

model_1.plot(label='Model')
plt.xlim(0, 120)
plt.yticks()
plt.gcf().set_size_inches(15, 4)


In order to plot my model against the data for German flu trends, both plots need to be on the same x-scale. Timestamp values in the date column will be converted into integers. Due to the running average, each value corresponds the the end of the month in intervals of two months, starting on the 31st-Jan-04.

As each time is on the last day of its corresponding month, I will consider each interval as being half way between its month and the next. 

For example, as January is coded as 0 on the scale, the first entry of 31/01 would be 0.5. The next entry is two months later on 31/03, which will be 2.5, followed by 4.5, 6.5, etc.

In [ ]:
germany_flu_scale = \
    (germany_flu['Germany']/max(germany_flu['Germany']))\
    .resample('2M', how='mean').reset_index().iloc[2:, :]

germany_flu_scale['date_int'] = np.arange(0.5, 142, 2)

germany_flu_scale.head()


In [ ]:
germany_flu_scale = \
    (germany_flu['Germany']/max(germany_flu['Germany']))\
    .resample('2M', how='mean').reset_index().iloc[2:, :]

germany_flu_scale['date_int'] = np.arange(0.5, 142, 2)

germany_flu_scale.head()


In [ ]:
print('Pearson correlation coefficient: ' +
      str(stats.pearsonr(model_1[0], germany_flu_scale['Germany'])[0]))

plt.plot(germany_flu_scale['date_int'], germany_flu_scale['Germany'])

plt.plot(model_1, label='Model 1')

plt.xlim(0, 120)
plt.yticks()
plt.legend()

plt.gcf().set_size_inches(15, 4)


Adjustments to the mathematical function were made to better fit the data. These modifications yielded a higher correlation coefficient, suggesting a closer relationship and therefore a better model.

In [ ]:
x = np.arange(0.5, 140.6, 2)
y = (np.sin(x/1.9 + 0.9) + 1.6) / 5

model_2 = pd.DataFrame(y, x)
print('Pearson correlation coefficient: ' +
      str(stats.pearsonr(model_2[0], germany_flu_scale['Germany'])[0]))

plt.plot(germany_flu_scale['date_int'], germany_flu_scale['Germany'])
plt.plot(model_2, label='Model 2')
plt.xlim(0, 120)
plt.yticks()
plt.legend()

plt.gcf().set_size_inches(15, 4)


Increasing the number of points plotted against the model gives a smoother curve.

This model reflects the general seasonal trends of flu in Germany. It does not, however, account for or predict accurately years, for which there is a spike in flu related searches or a lower than usual occurence. More data would be needed to search for a possible correlation that might suggest an answer and therefore some measure to predict these changes.

A loop could have been created to fine tune the equation in small increments to produce a more accurate curve. However, I believe that carrying this process out manually did suffice.

In [ ]:
x = np.arange(0.5, 140.6, 0.1)
y = (np.sin(x/1.9 + 0.9) + 1.6) / 5

plt.plot(germany_flu_scale['date_int'], germany_flu_scale['Germany'])
plt.plot(x, y, label='Model 2')

plt.xlim(0, 120)
plt.yticks()
plt.legend()
plt.title('Google Flu Trends From Germany and Predictive Model 2',
          fontdict=bold_dict)
plt.ylabel('Relative Frequency', fontdict=bold_dict)
plt.xlabel('Month From January 2004', fontdict=bold_dict)

plt.gcf().set_size_inches(15, 4)
